# Accés à l'API de Google Drive

In [8]:
# 1 Aller à https://console.cloud.google.com/ et créer un nouveau projet si inexistant
# 2 Créer des identifiants selectionner app user et apres web app
# 3 en sortie nommer ce fichier credentials.json et le placer dans le dossier notebooks


In [6]:
import os
import io
import pickle
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [ ]:

# Lecture seul
PERMISSIONS = ['https://www.googleapis.com/auth/drive.readonly']

def se_connecter():
    infos_connexion = None
    # Le fichier token.pickle permet de save  les infos de connexion
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as jeton:
            infos_connexion = pickle.load(jeton)
    if not infos_connexion or not infos_connexion.valid:
        # Si elles sont juste expirées mais qu'on peut les rafraîchir
        if infos_connexion and infos_connexion.expired and infos_connexion.refresh_token:
            #print("Jeton expiré, on rafraîchit...")
            infos_connexion.refresh(Request())
        else:
            #print(" on lance l'authentification...")
            flux = InstalledAppFlow.from_client_secrets_file('credentials.json', PERMISSIONS)
            # Ça va ouvrir une page dans le navigateur pour que l'utilisateur autorise l'appli.
            infos_connexion = flux.run_local_server(port=0)

        with open('token.pickle', 'wb') as jeton:
            pickle.dump(infos_connexion, jeton)
            
    service_drive = build('drive', 'v3', credentials=infos_connexion)
    print("Connexion à Google Drive réussie ")
    return service_drive

def list_fils(service_drive, id_dossier):
    """
    Récupère la liste des fichiers qui sont DANS le dossier spécifié par id_dossier.
    Ne regarde pas dans les sous-dossiers.
    """
    requete = f"'{id_dossier}' in parents"
    try:

        resultats = service_drive.files().list(q=requete, fields="files(id, name)").execute()
        # On récupère la liste des fichiers depuis la réponse.si ras, ça renvoie une liste vide [].
        fichiers = resultats.get('files', [])

        if not fichiers:
            print("-> Aucun fichier trouvé dans ce dossier.")
        else:
            print(f"-> Trouvé {len(fichiers)} fichier(s).")
        return fichiers
    except Exception as e:
        print(f" erreur en listant les fichiers : {e}")
        return [] 

def telecharger_fichier(service_drive, id_fichier, nom_fichier):
    """
    Télécharge UN fichier depuis Google Drive en utilisant son ID.
    Il sera sauvegardé sur l'ordinateur avec le nom 'nom_fichier'.
    """
    try:
        # On demande à Google Drive de nous donner le contenu du fichier (pas juste ses infos)
        demande_telechargement = service_drive.files().get_media(fileId=id_fichier)

        # On prépare un fichier sur notre ordinateur pour recevoir le contenu téléchargé.
        # 'wb' veut dire qu'on écrit en mode binaire (important pour les fichiers non-texte)
        fichier_local = io.FileIO(nom_fichier, 'wb')

        # On crée un objet pour gérer le téléchargement (surtout utile pour les gros fichiers,
        # ça télécharge par morceaux)
        telechargeur = MediaIoBaseDownload(fichier_local, demande_telechargement)

        termine = False
        while not termine:
            # On télécharge le prochain morceau
            etat, termine = telechargeur.next_chunk()
            if etat:
                # On affiche la progression
                print(f"Téléchargement de {nom_fichier} : {int(etat.progress() * 100)}% terminé.")

        #  fermer le fichier sur l'ordi une fois le téléchargement fini.
        fichier_local.close()
        print(f"Fichier '{nom_fichier}' téléchargé avec succès.")

    except Exception as e:
        print(f" impossible de télécharger {nom_fichier}. Erreur : {e}")
        # Si ça plante, on essaie de supprimer le fichier local potentiellement incomplet
        if os.path.exists(nom_fichier):
            try:
                os.remove(nom_fichier)
                print(f"Fichier local incomplet '{nom_fichier}' supprimé.")
            except OSError as err_suppr:
                print(f"Impossible de supprimer le fichier local incomplet '{nom_fichier}': {err_suppr}")

In [ ]:
id_dossier_a_scanner = 'nom_du_dossier' 
service = se_connecter()
alls_fils = list_fils(service, id_dossier_a_scanner)

In [ ]:
for fichier in alls_fils:
                print(f"\n--> Tentative de téléchargement de : {fichier['name']} (ID: {fichier['id']})")
                telecharger_fichier(service, fichier['id'], fichier['name'])
            print("\nTerminé ! Tous les fichiers ont été traités.")